## 0. Set up

In [1]:
!pip install datasets evaluate transformers
!pip install -q evaluate
!pip install -q rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
tor

In [2]:
from datasets import load_dataset
from evaluate import load
import re
import json
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from transformers import (AutoTokenizer, T5ForConditionalGeneration, Seq2SeqTrainer, Trainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, GPT2LMHeadModel,
                          DataCollatorForLanguageModeling, TrainingArguments)
import os
from tqdm import tqdm
os.environ["WANDB_DISABLED"] = "true"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## 1. Load and clean dataset

In [5]:
number_of_solutions = 1
def clean_dataset(batch):
  cleaned_questions = []
  cleaned_solutions = []
  difficulties = []

  for question, solutions_str, difficulty in zip(batch["question"], batch["solutions"], batch["difficulty"]):
      cleaned_question = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()!@:%_\+.~#?&\/\/=]*)', '', question)
      solutions = json.loads(solutions_str) if solutions_str  else []
      for sol in solutions[:number_of_solutions]:
          cleaned_questions.append(question)
          cleaned_solutions.append(sol)
          difficulties.append(difficulty)

  return {"question": cleaned_questions, "solution": cleaned_solutions, "difficulty": difficulties}

def tokenize_function_for_seq2seq(example, tokenizer): # tokenization function
    inputs = tokenizer(example['question'], truncation=True, max_length=512) # Tokenize inputs (questions)
    labels = tokenizer(example['solution'], truncation=True, max_length=512) # Tokenize targets (solutions)

    labels_with_ignore = [] # Replace padding token id with -100 so it's ignored in the loss
    for label in labels['input_ids']:
        labels_with_ignore.append([-100 if token == tokenizer.pad_token_id else token for token in label])
    inputs['labels'] = labels_with_ignore

    inputs['difficulty'] = example['difficulty']

    return inputs

In [6]:
cpa_dataset = load_dataset('codeparrot/apps')
cpa_test_set = cpa_dataset["test"]

cleaned_cpa_test_set = cpa_test_set.map(clean_dataset, batched=True, remove_columns=cpa_test_set.column_names)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.63k [00:00<?, ?B/s]

apps.py:   0%|          | 0.00/4.95k [00:00<?, ?B/s]

The repository for codeparrot/apps contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/codeparrot/apps.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


train.jsonl:   0%|          | 0.00/107M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/1.29G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

## 2. Load all models to evaluate

In [7]:
all_model_locs = {
    "T5-base": "Salesforce/codet5-base",
    "T5-base (finetuned)": '/content/drive/MyDrive/NLP_Code_Generation/t5-base-codeparrots', ## replace this with your fine-tuned model
    "GPT2-base": "gpt2",
    "GPT2-base (finetuned)": '/content/drive/MyDrive/NLP_Code_Generation/gpt2-codeparrots',  ## replaced this with your fidne-tuned model
}
### dictionary storing the model and tokenizer for each model for evaluation
all_models = {}
for model_name, model_loc in all_model_locs.items():
  all_models[model_name] = {}
  all_models[model_name]["tokenizer"] = AutoTokenizer.from_pretrained(model_loc)

  if model_name.startswith("T5"):
    all_models[model_name]["model"] = T5ForConditionalGeneration.from_pretrained(model_loc)

  elif model_name.startswith("GPT2"):
      all_models[model_name]["tokenizer"].pad_token = all_models[model_name]["tokenizer"].eos_token
      all_models[model_name]["model"] = GPT2LMHeadModel.from_pretrained(model_loc)

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## 3. Load and define evaluation metrics and utility functions

In [8]:
bleu = load('bleu')
rouge = load('rouge')
meteor = load('meteor')

def preprocess_logits_for_metrics_for_seq2seq(logits, labels):
    '''
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    https://discuss.huggingface.co/t/cuda-out-of-memory-when-using-trainer-with-compute-metrics/2941/15
    '''
    if isinstance(logits, tuple): pred_ids = logits[0]
    else: pred_ids = logits
    if pred_ids.ndim == 3: pred_ids = torch.argmax(pred_ids, dim=-1)
    return pred_ids, labels

def compute_metrics(eval_preds):
    preds = eval_preds.predictions[0]
    labels = eval_preds.label_ids
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id) # Replace -100 with pad token id
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id) # Replace -100 with pad token id

    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute BLEU, ROUGE, and exact match score
    bleu_results = bleu.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
    rouge_results = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    meteor_results = meteor.compute(predictions=decoded_preds, references=decoded_labels)
    # codebleu_results = codebleu.compute(predictions=decoded_preds, references=decoded_labels, lang="python")
    # exact_match = sum(pred == label for pred, label in zip(decoded_preds, decoded_labels)) / len(decoded_preds)

    return {
        'bleu': bleu_results['bleu'],
        'rouge1': rouge_results['rouge1'],
        'rouge2': rouge_results['rouge2'],
        'rougeL': rouge_results['rougeL'],
        'meteor': meteor_results['meteor'],
        # 'codebleu': meteor_results['codebleu'],
        # 'exact_match': exact_match,
    }

# Create eval trainer for seq2seq model
def create_eval_trainer_for_seq2seq(model, tokenizer, predict_with_generate=True):
  eval_args = Seq2SeqTrainingArguments(
      per_device_eval_batch_size=128,         # Evaluation batch size
      predict_with_generate=predict_with_generate,          # Whether to use generate to calculate generative metrics (ROUGE, BLEU)
      generation_max_length=256,
      fp16=torch.cuda.is_available(),       # Enable mixed-precision training if a CUDA GPU is available (faster, less memory)
      report_to=[]                          # disable wandb report
  )

  eval_trainer = Seq2SeqTrainer(
      model=model,
      args=eval_args,
      data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model), # Set up data collator for dynamic padding
      preprocess_logits_for_metrics=preprocess_logits_for_metrics_for_seq2seq,
      compute_metrics=compute_metrics,
  )
  return eval_trainer

# Evaluation functions seq2seq model: use Trainer.evaluate #
def evaluate_seq2seq_model(model, tokenizer, test_dataset):
  tokenized_test_set= test_dataset.map(lambda x: tokenize_function_for_seq2seq(x, tokenizer), batched=True, remove_columns=test_dataset.column_names)
  ## Create trainer
  eval_trainer = create_eval_trainer_for_seq2seq(model, tokenizer, predict_with_generate=True)
  metrics = eval_trainer.evaluate(tokenized_test_set, metric_key_prefix='test')
  return metrics

def tokenize_function_for_auto_regressive_evaluation(example, tokenizer): # tokenization function
    inputs = tokenizer(example['question'][:500] + '\n[CODE]\n', truncation=True, max_length=512)
    solution = tokenizer(example['solution'][:500] + tokenizer.eos_token, truncation=True, max_length=512)

    # Create labels - we only want to compute loss on the solution part
    labels = solution['input_ids']

    inputs['labels'] = labels
    return inputs

def shift_padding_to_left(batch, pad_token=-100):
    shifted_batch = []

    for sample in batch:
        # Find the index of the last non-padding token
        last_non_pad_idx = (sample != pad_token).nonzero(as_tuple=True)[0].max().item() if (sample != pad_token).any() else -1

        # Create a new tensor with padding shifted to the left
        shifted_sample = torch.full_like(sample, pad_token)  # Start with all padding (-100)
        shifted_sample[-(last_non_pad_idx + 1):] = sample[:last_non_pad_idx + 1]  # Copy valid tokens to the right side

        shifted_batch.append(shifted_sample)

    return torch.stack(shifted_batch)

# Evaluation functions for autoregressive #
def evaluate_autoregressive_model(model, tokenizer, test_dataset):
  tokenized_test_set= test_dataset.map(lambda x: tokenize_function_for_auto_regressive_evaluation(x, tokenizer), batched=False, remove_columns=test_dataset.column_names)
  eval_dataloader = DataLoader(tokenized_test_set, shuffle=False, batch_size=16,
                               collate_fn=DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True))
  model.to(device)
  model.eval()
  decoded_preds = []
  labels = []
  for batch in tqdm(eval_dataloader):
    input_ids = batch['input_ids']
    ## Reverse the pad token to the left for proper generation
    input_ids = shift_padding_to_left(input_ids, tokenizer.pad_token_id)

    ## process label to remove -100 for decoding
    mask = batch['labels'] != -100
    ragged_list = [sample[mask_row].tolist() for sample, mask_row in zip(batch['labels'], mask)]
    label = tokenizer.batch_decode(ragged_list)

    with torch.no_grad():
        gen_ids = model.generate(
            input_ids.to(device),
            do_sample=True,
            temperature=0.9,
            max_new_tokens=512,
            pad_token_id=tokenizer.pad_token_id,
            attention_mask=batch['attention_mask'].to(device),
            eos_token_id=tokenizer.eos_token_id
        )

        gen_ids = gen_ids[:, input_ids.shape[-1]:]

    gen_text = tokenizer.batch_decode(gen_ids.to('cpu'))
    decoded_preds.extend(gen_text)
    labels.extend(label)

  bleu_results = bleu.compute(predictions=decoded_preds, references=[[label] for label in labels])
  rouge_results = rouge.compute(predictions=decoded_preds, references=labels)
  meteor_results = meteor.compute(predictions=decoded_preds, references=labels)

  return {
      'test_bleu': bleu_results['bleu'],
      'test_rouge1': rouge_results['rouge1'],
      'test_rouge2': rouge_results['rouge2'],
      'test_rougeL': rouge_results['rougeL'],
      'test_meteor': meteor_results['meteor'],
  }

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


## 4. Run evaluation

### 4.1. Comparison between different Seq2Seq on the entire test set (different difficulty levels)

In [ ]:
model_comparison_results_df = pd.DataFrame(columns=['model', 'difficulty', 'bleu', 'rouge1', 'rouge2', 'rougeL', 'meteor'])

all_difficulty_lvls = ['all'] + list(set(cleaned_cpa_test_set['difficulty']))
def select_samples_based_on_difficulties(dataset, difficulty='all'):
  return dataset if (difficulty == 'all') else dataset.filter(lambda example: example['difficulty'] == difficulty)

for model_name in ["T5-base (finetuned)", "T5-base"]:
  print(f"Evaluating model: {model_name}")
  model = all_models[model_name]["model"]
  tokenizer = all_models[model_name]["tokenizer"]

  ## for each difficulty levels
  for lvl in all_difficulty_lvls:
    # Filter out the test set to only the current level
    test_dataset_lvl = select_samples_based_on_difficulties(cleaned_cpa_test_set, difficulty=lvl)

    ## Run evaluation
    if model_name.startswith("T5"):
      metrics = evaluate_seq2seq_model(model, tokenizer, test_dataset_lvl)
    elif model_name.startswith("GPT"):
      metrics = evaluate_autoregressive_model(model, tokenizer, test_dataset_lvl)


    model_comparison_results_df.loc[len(model_comparison_results_df)] = [model_name, lvl, metrics['test_bleu'], metrics['test_rouge1'], metrics['test_rouge2'], metrics['test_rougeL'], metrics['test_meteor']]


model_comparison_results_df = model_comparison_results_df.pivot(index="model", columns="difficulty", values=["bleu", "rouge1", "rouge2", "rougeL", "meteor"])
model_comparison_results_df = model_comparison_results_df.sort_index(axis=1, level=1)
model_comparison_results_df.columns = pd.MultiIndex.from_tuples([(d, m) for m, d in model_comparison_results_df.columns])
model_comparison_results_df

Evaluating model: T5-base (finetuned)


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Filter:   0%|          | 0/3765 [00:00<?, ? examples/s]

Map:   0%|          | 0/728 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3765 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/3765 [00:00<?, ? examples/s]

Map:   0%|          | 0/728 [00:00<?, ? examples/s]

Map:   0%|          | 0/310 [00:00<?, ? examples/s]

Map:   0%|          | 0/2727 [00:00<?, ? examples/s]

all                                          \
                         bleu    meteor    rouge1    rouge2    rougeL   
model                                                                   
T5-base              0.000027  0.010727  0.036318  0.002785  0.033925   
T5-base (finetuned)  0.143529  0.247180  0.317829  0.143645  0.266808   

                    competition                                          \
                           bleu    meteor    rouge1    rouge2    rougeL   
model                                                                     
T5-base                0.000003  0.009627  0.027273  0.002109  0.025698   
T5-base (finetuned)    0.105458  0.202040  0.296873  0.125648  0.242936   

                    interview                                          \
                         bleu    meteor    rouge1    rouge2    rougeL   
model                                                                   
T5-base              0.000017  0.010241  0.034132  0.002344  0.031817   
T5-base (finetuned)  0.139844  0.242647  0.316935  0.141857  0.264480   

                    introductory                                          
                            bleu    meteor    rouge1    rouge2    rougeL  
model                                                                     
T5-base                 0.000177  0.012921  0.048156  0.004595  0.044805  
T5-base (finetuned)     0.182422  0.283353  0.330613  0.158015  0.285584

### 4.2. Comparison between Autoregressive model

In [10]:
model_comparison_results_df = pd.DataFrame(columns=['model', 'difficulty', 'bleu', 'rouge1', 'rouge2', 'rougeL', 'meteor'])

# all_difficulty_lvls = ['all'] + list(set(cleaned_cpa_test_set['difficulty']))
all_difficulty_lvls = list(set(cleaned_cpa_test_set['difficulty']))
# only evaluate on all diff levels and the first 1500 test samples
# all_difficulty_lvls = ['all']
# cleaned_cpa_test_set = cleaned_cpa_test_set.select(range(1500))

def select_samples_based_on_difficulties(dataset, difficulty='all'):
  return dataset if (difficulty == 'all') else dataset.filter(lambda example: example['difficulty'] == difficulty)

for model_name in ["GPT2-base (finetuned)", "GPT2-base"]:
  print(f"Evaluating model: {model_name}")
  model = all_models[model_name]["model"]
  tokenizer = all_models[model_name]["tokenizer"]

  ## for each difficulty levels
  for lvl in all_difficulty_lvls:
    # Filter out the test set to only the current level
    test_dataset_lvl = select_samples_based_on_difficulties(cleaned_cpa_test_set, difficulty=lvl)

    ## Run evaluation
    if model_name.startswith("T5"):
      metrics = evaluate_seq2seq_model(model, tokenizer, test_dataset_lvl)
    elif model_name.startswith("GPT"):
      metrics = evaluate_autoregressive_model(model, tokenizer, test_dataset_lvl)

    model_comparison_results_df.loc[len(model_comparison_results_df)] = [model_name, lvl, metrics['test_bleu'], metrics['test_rouge1'], metrics['test_rouge2'], metrics['test_rougeL'], metrics['test_meteor']]


model_comparison_results_df = model_comparison_results_df.pivot(index="model", columns="difficulty", values=["bleu", "rouge1", "rouge2", "rougeL", "meteor"])
model_comparison_results_df = model_comparison_results_df.sort_index(axis=1, level=1)
model_comparison_results_df.columns = pd.MultiIndex.from_tuples([(d, m) for m, d in model_comparison_results_df.columns])
model_comparison_results_df

Evaluating model: GPT2-base (finetuned)


Filter:   0%|          | 0/3765 [00:00<?, ? examples/s]

Map:   0%|          | 0/2727 [00:00<?, ? examples/s]

100%|██████████| 171/171 [28:40<00:00, 10.06s/it]


Filter:   0%|          | 0/3765 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/728 [00:00<?, ? examples/s]

100%|██████████| 46/46 [07:17<00:00,  9.52s/it]


Map:   0%|          | 0/310 [00:00<?, ? examples/s]

100%|██████████| 20/20 [03:08<00:00,  9.41s/it]


competition                                          \
                             bleu    meteor    rouge1    rouge2    rougeL   
model                                                                       
GPT2-base                0.003547  0.080809  0.062215  0.003525  0.045240   
GPT2-base (finetuned)    0.077413  0.274746  0.246234  0.064276  0.159001   

                      interview                                          \
                           bleu    meteor    rouge1    rouge2    rougeL   
model                                                                     
GPT2-base              0.003494  0.076321  0.053773  0.002651  0.040205   
GPT2-base (finetuned)  0.075023  0.271041  0.230575  0.064646  0.153964   

                      introductory                                          
                              bleu    meteor    rouge1    rouge2    rougeL  
model                                                                       
GPT2-base                 0.003267  0.080458  0.050316  0.003519  0.038521  
GPT2-base (finetuned)     0.069472  0.270501  0.210595  0.065054  0.146088

In [12]:
model_comparison_results_df.to_excel("/content/drive/MyDrive/auto_regressive_comparisons.xlsx")

### 4.3. Comparison on different decoding method

In [ ]:
selected_model_name = "T5-base (finetuned)" ## you can set it to another model ##
model = all_models[selected_model_name]["model"]
tokenizer = all_models[selected_model_name]["tokenizer"]

### Tokenization ###
if selected_model_name.startswith("T5"):
  tokenized_test_set= cleaned_cpa_test_set.map(lambda x: tokenize_function_for_seq2seq(x, tokenizer), batched=True, remove_columns=cleaned_cpa_test_set.column_names)

eval_trainer = create_eval_trainer_for_seq2seq(model, tokenizer, predict_with_generate=True)

Map:   0%|          | 0/3765 [00:00<?, ? examples/s]

In [ ]:
### evaluate with different generation schema
generation_schemas =  {
    "greedy": {"do_sample": False, "num_beams": 1},
    "beam_search": {"do_sample": False, "num_beams": 5, "early_stopping": True},
    "random_sampling": {"do_sample": True, "temperature": 0.6, "top_k": 0},
    "top_k_sampling": {"do_sample": True, "top_k": 50},
    "top_p_sampling": {"do_sample": True, "top_p": 0.9, "top_k": 0},
}

schema_results_df = pd.DataFrame(columns=["generation_method", "bleu", "rouge1", "rouge2", "rougeL", "meteor"])
for schema in generation_schemas:
  schema_config = generation_schemas[schema]
  metrics = eval_trainer.predict(tokenized_test_set ,metric_key_prefix="test", **schema_config).metrics
  schema_results_df.loc[len(schema_results_df)] = [schema, metrics['test_bleu'], metrics['test_rouge1'], metrics['test_rouge2'], metrics['test_rougeL'], metrics['test_meteor']]

schema_results_df

,generation_method,bleu,rouge1,rouge2,rougeL,meteor
0,greedy,0.143529,0.317829,0.143645,0.266808,0.247180
1,beam_search,0.151078,0.314411,0.138405,0.262196,0.254163
2,random_sampling,0.155126,0.340696,0.141335,0.266654,0.263644
3,top_k_sampling,0.152932,0.332511,0.116916,0.239430,0.270980
4,top_p_sampling,0.158862,0.335466,0.124047,0.247006,0.273418


In [ ]:
# model_comparison_results_df.to_excel("model_comparison_results_df.xlsx")
# schema_results_df.to_excel("schema_results_df.xlsx")